In [1]:
## header file function (need to be script )
import sys
import numpy as np
import vireoSNP
from vireoSNP.utils.vireo_base import match
from vireoSNP.utils.vcf_utils import load_VCF, write_VCF, parse_donor_GPb
from vireoSNP.utils.vcf_utils import read_sparse_GeneINFO, GenoINFO_maker
import matplotlib.pyplot as plt
import pandas as pd
from vireoSNP.utils.io_utils import match_donor_VCF
import re as re
def VCF_to_sdf(bulk_data):
    df_empty=pd.DataFrame(columns=["variants","AD","DP","GT"])
    bulk_dat = read_sparse_GeneINFO(bulk_data['GenoINFO'], keys=['AD', 'DP'])
    df_empty["variants"]=bulk_data["variants"]
    df_empty["AD"]=bulk_dat["AD"].toarray()
    df_empty["DP"]=bulk_dat["DP"].toarray()
    return (df_empty)

def rematch_vcf(bulk_data,donor_vcf):
    df_bulk=VCF_to_sdf(bulk_data)
    df_donor=pd.DataFrame(columns=["variants","GT"])
    df_donor["variants"]=donor_vcf["variants"]
    df_donor["GT"]=donor_vcf["GenoINFO"]["GT"]     
    donor_tensor = vireoSNP.vcf.parse_donor_GPb(donor_vcf['GenoINFO']["GT"], "GT")
    itersect_variants=pd.Series(list(set(df_donor["variants"]).intersection(set(df_bulk["variants"]))))
    id_d=df_donor["variants"].isin(itersect_variants)
    id_b=df_bulk["variants"].isin(itersect_variants)
    df_bulk=df_bulk[id_b]
    df_donor=df_donor[id_d]
    donor_tensor=donor_tensor[id_d]
    return(df_bulk,df_donor,donor_tensor)
def rematch_vcf_annotated(bulk_data,donor_vcf,donor_vcf_anno):
    df_bulk=VCF_to_sdf(bulk_data)
    df_donora=pd.DataFrame(columns=["variants","function","genes","avsnp","GT"])
    df_donora["variants"]=donor_vcf_anno["variants"]  
    df_donora["function"]=donor_vcf_anno["function."]
    df_donora["genes"]=donor_vcf_anno["genes"]
    df_donora["avsnp"]=donor_vcf_anno["avsnp"]
    df_donor=pd.DataFrame(columns=["variants","GT"])
    df_donor["variants"]=donor_vcf["variants"]
    df_donor["GT"]=donor_vcf["GenoINFO"]["GT"]     
    donor_tensor = vireoSNP.vcf.parse_donor_GPb(donor_vcf['GenoINFO']["GT"], "GT")
    itersect_variants=pd.Series(list(set(df_donora["variants"]).intersection(set(df_bulk["variants"]))))
    id_d=df_donor["variants"].isin(itersect_variants)
    id_b=df_bulk["variants"].isin(itersect_variants)
    id_da=df_donora["variants"].isin(itersect_variants)
    df_bulk=df_bulk[id_b]
    df_donor=df_donor[id_d]
    df_donora=df_donora[id_da]
    donor_tensor=donor_tensor[id_d]
    df_donora["GT"]=df_donora["GT"]=df_donor["GT"].tolist()
    return(df_bulk,df_donora,donor_tensor)
def merged_VCF_to_sdf(bulk_data_merge):
    df_empty=pd.DataFrame(columns=["variants","AD","DP","OTH"])
    df_empty["variants"]=bulk_data_merge["variants"]
    c=np.array(bulk_data_merge['FixedINFO']["INFO"])
    for i in range(len(c)):
        tem=c[i].tolist().split(";",-1)
        df_empty["AD"][i]=float(re.sub("AD=","",tem[0]))
        df_empty["DP"][i]=float(re.sub("DP=","",tem[1]))
        df_empty["OTH"][i]=float(re.sub("OTH=","",tem[2]))

    return(df_empty)
def rematch_merge_vcf_annotated(bulk_data,donor_vcf,donor_vcf_anno):
    df_bulk=merged_VCF_to_sdf(bulk_data)
    df_donora=pd.DataFrame(columns=["variants","function","genes","avsnp","GT"])
    df_donora["variants"]=donor_vcf_anno["variants"]  
    df_donora["function"]=donor_vcf_anno["function."]
    df_donora["genes"]=donor_vcf_anno["genes"]
    df_donora["avsnp"]=donor_vcf_anno["avsnp"]
    df_donor=pd.DataFrame(columns=["variants","GT"])
    df_donor["variants"]=donor_vcf["variants"]
    df_donor["GT"]=donor_vcf["GenoINFO"]["GT"]     
    donor_tensor = vireoSNP.vcf.parse_donor_GPb(donor_vcf['GenoINFO']["GT"], "GT")
    itersect_variants=pd.Series(list(set(df_donora["variants"]).intersection(set(df_bulk["variants"]))))
    id_d=df_donor["variants"].isin(itersect_variants)
    id_b=df_bulk["variants"].isin(itersect_variants)
    id_da=df_donora["variants"].isin(itersect_variants)
    df_bulk=df_bulk[id_b]
    df_donor=df_donor[id_d]
    df_donora=df_donora[id_da]
    donor_tensor=donor_tensor[id_d]
    df_donora["GT"]=df_donora["GT"]=df_donor["GT"].tolist()
    return(df_bulk,df_donora,donor_tensor)
def find_all(data, s):
    r_list = []
    for r in range(len(data)):
        r_list.append( s in data[r])
 
    return(r_list)
def removedfault(df_bulk,df_donora,donor_tensor):
    id_d=[]
    for bool in find_all(np.array(df_donora["GT"]).tolist(),"./."):
        id_d.append(not bool)
    df_donor=df_donora[id_d]
    df_gt=donor_tensor[id_d]
    id_b=df_bulk["variants"].isin(df_donor["variants"])
    df_b=df_bulk[id_b]
    return(df_b,df_donor,df_gt)

In [17]:
def prediction_bulk(df_b,df_d,df_gt,donor_vcf,pre_model,genelist):
    gene_unique=genelist
    new=pd.DataFrame(index=gene_unique,columns=donor_vcf['samples']+["chi","p","numSNP"])
    pre_psi=pre_model
    test_d=df_d
    test_b=df_b
    test_gt=df_gt
    sample_list=donor_vcf['samples']
    nk=len(donor_vcf['samples'])
    for i in range(len(gene_unique)):
        sub_d=test_d[test_d["genes"]==gene_unique[i]]
        sub_b=test_b[test_b["variants"].isin(test_d[test_d["genes"]==gene_unique[i]]["variants"])]
        sub_gt=test_gt[test_d["genes"]==gene_unique[i]]
        a=np.array(sub_b["AD"])
        d=np.array(sub_b["DP"])
        a=np.array(list(map(float,a)))
        d=np.array(list(map(float,d)))
        model =vireoSNP.VireoBulk(nk)
        model.fit(a,d,sub_gt,learn_theta=False )
        pv=vireoSNP.LikRatio_test(model.psi,pre_psi,a,d,sub_gt,model.theta)
        for j in range(len(model.psi)):
            new[sample_list[j]][i]=model.psi[j]
        new["chi"][i]=pv[0]
        new["p"][i]=pv[1]
        new["numSNP"][i]=len(a)
    return(new)

In [4]:
donor_vcf=load_VCF(   "/media/wgs/subset_stimulation_bam/filter_pbmc10donors.vcf.gz",    biallelic_only=True,    load_sample=True, sparse=False , format_list=None)
####count based model
##bulk_merged_data=load_VCF(   "/media/wgs/subset_stimulation_bam/pbmc10donor_bulk/cellSNP.cells.vcf.gz",    biallelic_only=True,    load_sample=True,    format_list=None,)
#### UMI based model
bulk_merged_data=load_VCF("/media/wgs/subset_stimulation_bam/pbmc10donor/cellSNP.base.vcf.gz", biallelic_only=True,load_sample=False,format_list=None,)
###processedcsv
df_anno=pd.read_csv("/media/wgs/subset_stimulation_bam/pbmc10donor/scvcf_annotated_processed.csv")

In [ ]:
#count_bulk_data
count_bulk_data=load_VCF(   "/media/wgs/subset_stimulation_bam/pbmc10donor_bulk/cellSNP.cells.vcf.gz",    biallelic_only=True,    load_sample=True,    format_list=None,)
cf_b,cf_d,cm_GT=rematch_merge_vcf_annotated(count_bulk_data,donor_vcf,df_anno)
cf_b,cf_d,cm_GT=removedfault(cf_b,cf_d,cm_GT)

In [5]:
df_b,df_d,m_GT=rematch_merge_vcf_annotated(bulk_merged_data,donor_vcf,df_anno)

In [6]:
df_b,df_d,m_GT=removedfault(df_b,df_d,m_GT)

In [7]:
m_GT.shape

(127155, 10, 3)

In [8]:
gene_unique=np.unique(df_d["genes"].tolist())

In [9]:
a=np.array(df_b["AD"])
d=np.array(df_b["DP"])
a=np.array(list(map(float,a)))
d=np.array(list(map(float,d)))

In [10]:
model =vireoSNP.VireoBulk(10)

In [11]:
model.fit(a,d,m_GT,learn_theta=True )

In [12]:
model.theta

array([0.01055865, 0.44923362, 0.99271496])

In [13]:
donor_vcf['samples']

['18-G-017_R1',
 '18-G-018_R1',
 '18-G-019_R1',
 '18-G-020_R1',
 '18-G-021_R1',
 '18-G-022_R1',
 '18-G-023_R1',
 '18-G-024_R1',
 '18-G-025_R1',
 '18-G-026_R1']

In [14]:
premodel=model.psi

In [18]:
bulk_demuti_result= prediction_bulk(df_b,df_d,m_GT,donor_vcf,premodel,gene_unique)

In [19]:
bulk_demuti_result

,18-G-017_R1,18-G-018_R1,18-G-019_R1,18-G-020_R1,18-G-021_R1,18-G-022_R1,18-G-023_R1,18-G-024_R1,18-G-025_R1,18-G-026_R1,chi,p,numSNP
A1BG,0.204566,0.006103,0.070499,0.033077,0.435507,0.029794,0.084591,0.091253,0.002081,0.042529,2.975999,0.965239,3
A1BG-AS1,0.106685,0.092529,0.070027,0.073083,0.055836,0.026301,0.001112,0.061273,0.323471,0.189682,3.044456,0.962506,2
A2M,0.307062,0.005625,0.081138,0.002078,0.004711,0.007919,0.206134,0.228321,0.137182,0.01983,6.902324,0.647288,2
A2M;PZP,0.329138,0.1057,0.185051,0.060067,0.021698,0.009457,0.238751,0.017388,0.002324,0.030427,5.998282,0.74009,1
A2MP1,0.046441,0.467386,0.03681,0.029199,0.056147,0.089542,0.137729,0.094189,0.042559,0.0,23.019619,0.006152,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZXDC,0.098057,0.041216,0.135931,0.247421,0.032694,0.012474,0.17658,0.163637,0.034342,0.057647,2.058528,0.990508,13
ZYG11B,0.021099,0.459046,0.135734,0.012327,0.133187,0.017391,0.05394,0.0116,0.145402,0.010273,10.922011,0.281093,9
ZYX,0.026786,0.084729,0.049946,0.189908,0.073098,0.163856,0.047506,0.135261,0.223413,0.005498,3.735993,0.927914,2
ZZEF1,0.012283,0.004326,0.134847,0.117929,0.272921,0.105438,0.054967,0.005739,0.190282,0.101267,1.994955,0.991547,29


In [20]:
bulk_demuti_result.to_csv("/media/wgs/subset_stimulation_bam/pbmc10donor/annotated_expo_umi2.csv")